In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 0: Load the dataset
games_data = pd.read_csv("/games.csv")  # Replace with your actual file path

# Step 1: Prepare features and target
features = ['HDCF%', 'xGF%', 'FF%', 'CF%', 'SCF%', 'PDO']
X = games_data[features].fillna(0)  # Ensure no missing values
y = (games_data['W'] > games_data['L']).astype(int)  # Target: 1 for win, 0 for loss

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 2: Train the model
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 3: Define the prediction function
def predict_matchup():
    """
    Ask the user to input two teams and predict the winner using the trained model.
    """
    # Input team names
    team1 = input("Enter the name of the first team: ").strip()
    team2 = input("Enter the name of the second team: ").strip()

    # Get stats for both teams
    team1_stats = games_data[games_data['Team'] == team1][features]
    team2_stats = games_data[games_data['Team'] == team2][features]

    # Check if the teams are in the dataset
    if team1_stats.empty or team2_stats.empty:
        print(f"One or both team names ('{team1}', '{team2}') not found in the dataset.")
        return

    # Normalize input stats
    matchup_stats = pd.concat([team1_stats.mean(), team2_stats.mean()], axis=1).T.fillna(0)
    matchup_stats_scaled = scaler.transform(matchup_stats)

    # Predict probabilities
    probabilities = model.predict_proba(matchup_stats_scaled)
    team1_prob = max(min(probabilities[0][1], 0.99), 0.01)
    team2_prob = 1 - team1_prob

    # Determine winner
    winner = team1 if team1_prob > team2_prob else team2
    print(f"The model predicts '{winner}' to win.")
    print(f"Win Probabilities:\n  {team1}: {team1_prob:.2%}\n  {team2}: {team2_prob:.2%}")

# Step 4: Interactive Loop
def main():
    print("Welcome to the NHL Matchup Predictor!")
    while True:
        predict_matchup()
        cont = input("Do you want to predict another matchup? (yes/no): ").strip().lower()
        if cont != "yes":
            print("Thank you for using the NHL Matchup Predictor!")
            break

# Run the program
if __name__ == "__main__":
    main()


Welcome to the NHL Matchup Predictor!
Enter the name of the first team: Carolina Hurricanes
Enter the name of the second team: New Jersey Devils
The model predicts 'Carolina Hurricanes' to win.
Win Probabilities:
  Carolina Hurricanes: 93.00%
  New Jersey Devils: 7.00%
Do you want to predict another matchup? (yes/no): no
Thank you for using the NHL Matchup Predictor!
